In [ ]:
#from werkzeug.security import generate_password_hash
#print(generate_password_hash("test"))

In [18]:
import sqlite3
import pandas as pd

In [19]:
db_file = "database.db"

In [20]:
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

In [15]:
#create munros table
create_munro_table = """CREATE TABLE munros (
                    munro_id TEXT PRIMARY KEY,
                    name TEXT UNIQUE,
                    region TEXT,
                    description TEXT,
                    whl_url TEXT,
                    latitude FLOAT,
                    longitude FLOAT,
                );
                """
cursor.execute("DROP TABLE IF EXISTS munros;"")
cursor.execute(create_munro_table)

#add data from csv
munro_df = pd.read_csv("munros_table.csv")
munro_df.to_sql("munros", conn, if_exists="append", index=False)
conn.commit()

In [26]:
#create users table
create_user_table = """CREATE TABLE users (
                    user_id TEXT PRIMARY KEY,
                    user_name TEXT UNIQUE NOT NULL,
                    password TEXT NOT NULL
                );
                """

cursor.execute("DROP TABLE IF EXISTS users;")
cursor.execute(create_user_table)

#delete all users
cursor.execute("DELETE FROM users;")
conn.commit()

#add users from df
users = ["matthew", "amy", "scott", "emily",]

user_df = pd.DataFrame({
                "user_id": [f"u{i:03}" for i in range(len(users))],
                "user_name": users,
                "password": 123,
                })

#need to use append so not to change table schema
user_df.to_sql("users", conn, if_exists="append", index=False)
conn.commit()

In [27]:
#create junction table
#add junction bag table
create_junction_table = """
                        CREATE TABLE bags (
                        user_id INTEGER NOT NULL,
                        munro_id INTEGER NOT NULL,
                        date TEXT NOT NULL,
                        distance TEXT,
                        friends TEXT,
                        notes TEXT,
                        FOREIGN KEY (user_id) REFERENCES users(user_id),
                        FOREIGN KEY (munro_id) REFERENCES munros(munro_id)
                        );
                        """

#cursor.execute("DROP TABLE IF EXISTS bags;")
#cursor.execute(create_junction_table)
#conn.commit()

In [23]:
#add bags from csv
#my_df = pd.read_csv("C:/Users/mrallison/Downloads/munros - amy_bags.csv", dtype=str, keep_default_na=True)
amy_df.to_sql('bags', conn, if_exists='append', index=False)

35

In [3]:
#matthew_df = pd.read_csv("C:/Users/mrallison/Downloads/munros - Sheet7.csv", dtype=str)
#matthew_df["distance"] = None
#matthew_df["friends"] = None
#matthew_df["notes"] = None

#matthew_df.to_sql('bags', conn, if_exists='append', index=False)

In [5]:
matthew_df["date"][0]

'2024-06-22'

In [9]:
amy_df["date"][0]

'9/7/2019'

In [15]:
def date_str(date):

    day, month, year = date.split("/")

    if len(day) == 1:
        day = "0"+day
    if len(month) == 1:
        month = "0"+month
        

    return f"{year}-{month}-{day}"

In [16]:
amy_df["date"] = amy_df["date"].apply(lambda x: date_str(x))

In [22]:
del_query = "DELETE FROM bags WHERE user_id = 'u001';"
cursor.execute(del_query)

In [24]:
conn.commit()

In [25]:
alt_query = """ALTER TABLE bags
ALTER COLUMN date TYPE DATE
USING to_date(date, 'YYYY-MM-DD');"""

In [26]:
cursor.execute(alt_query)

OperationalError: near "ALTER": syntax error